# Metrics Comparisons

In [1]:
import sys
sys.path.append('..')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from pathlib import Path
from ipywidgets import SelectMultiple, FloatRangeSlider, interact, fixed

%matplotlib inline

from gerumo.visualization import metrics


## Configuration

In [8]:
root_experiments_folder = '/mnt/storage-lite/experiments/cta/angular'

experiments_include_patterns = [
    'smooth_experiments/*_lst_*_l2*/evaluation/*/results.csv'
]

## Load experiments

In [9]:
experiments = defaultdict(dict)
eval_datasets = set()
for pattern in experiments_include_patterns:
    for results_filepath in Path(root_experiments_folder).rglob(pattern):
        *_, experiment_name, _, eval_dataset, _ = str(results_filepath).split('/') 
        eval_datasets.add(eval_dataset)
        experiments[experiment_name][eval_dataset] = pd.read_csv(results_filepath)

In [10]:
def get_label(experiment_name, dataset):
    alias = {
        'test_g_cut1000': 'on-axis (cut)', 
        'test_gd_cut1000': 'off-axis (cut)',
        'test_gd_full': 'off-axis'
    }
    return f'{experiment_name.split("_", 3)[-1]} -  {alias[dataset]}'

In [11]:
@interact(
    experiments_names=SelectMultiple(
        options=experiments.keys(),
        rows=10, description='Models & Ensembles'),
    datasets=SelectMultiple(
        options=list(eval_datasets),
        rows=4, description='Datasets'),
    ylim=FloatRangeSlider(
        value=[0, 1.0], min=0, max=2.0, step=0.1, description='Y limits:', continuous_update=False, orientation='horizontal',
        readout=True, readout_format='.1f'
    ),
    xlim=FloatRangeSlider(
        value=[-2, 2], min=-2.0, max=2.0, step=0.1, description='X limits:', continuous_update=False, orientation='horizontal',
        readout=True, readout_format='.1f'
    )
)
def plot_angular_comparison(experiments_names, datasets, ylim=(0, 2), xlim=(-2,2)):
    if len(experiments_names) == 0: return
    if len(datasets) == 0: return
    log_xlim = (10**xlim[0], 10**xlim[1])
    #fmts = [formats[model] for model in models]
    #fmts = list(fmts)
    global experiments
    figure = plt.figure(figsize=(10, 10))
    for experiment_name in experiments_names:
        for dataset in datasets:
            results = experiments[experiment_name].get(dataset, None)
            if results is not None:
                metrics.reconstruction_resolution(results, ['az', 'alt'], ylim=ylim, xlim=log_xlim, figure=figure, label=get_label(experiment_name, dataset))#, fmts=fmts)
    plt.show()

interactive(children=(SelectMultiple(description='Models & Ensembles', options=('20220604_072240_umonne_lst_cu…